In [2]:
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt

from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

In [3]:
import rasterio
import rasterio.plot as rioplot

In [4]:
import datetime
from pyproj import CRS
import xarray as xr
import rioxarray
import regionmask as rm

In [4]:
datos_ruta = Path("~/shared/proyecto5")

In [5]:
ecoregiones_gdf = gpd.read_file(datos_ruta / "MEOW" / "meow_ecos.shp")

Seleccionamos la ecoregio

In [6]:
def seleccion_ecoregion(gdf, ecoregion):
    sel_gdf = gdf[gdf["ECOREGION"] == ecoregion]
    sel_bordes_df = sel_gdf.geometry.bounds
    sel_bordes = dict(
        minx=sel_bordes_df.minx.values[0],
        maxx=sel_bordes_df.maxx.values[0],
        miny=sel_bordes_df.miny.values[0],
        maxy=sel_bordes_df.maxy.values[0],
    )
    return sel_gdf, sel_bordes

In [7]:
ecoregion_gdf, ecoregion_bordes = seleccion_ecoregion(ecoregiones_gdf, "Central Chile")

ecoregion_bordes
Leer los datos ERDDAP

In [8]:
erddap_opendap_url = 'https://coastwatch.pfeg.noaa.gov/erddap/es/griddap/erdMH1chlamday_R2022SQ'

fuente_ds = xr.open_dataset(erddap_opendap_url)

In [9]:
fuente_ds

<xarray.Dataset>
Dimensions:    (time: 245, latitude: 4320, longitude: 8640)
Coordinates:
  * time       (time) datetime64[ns] 2002-07-16 2002-08-16 ... 2022-11-16
  * latitude   (latitude) float32 89.98 89.94 89.9 89.85 ... -89.9 -89.94 -89.98
  * longitude  (longitude) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
Data variables:
    chlor_a    (time, latitude, longitude) float32 ...
Attributes: (12/52)
    _lastModified:                     2023-01-31T21:02:35.000Z
    cdm_data_type:                     Grid
    Conventions:                       CF-1.6 ACDD-1.3, COARDS
    creator_email:                     data@oceancolor.gsfc.nasa.gov
    creator_name:                      NASA/GSFC/OBPG
    creator_type:                      group
    ...                                ...
    summary:                           Moderate Resolution Imaging Spectrorad...
    temporal_range:                    month
    time_coverage_end:                 2022-11-16T00:00:00Z
    time_coverage_start:               2002-07-16T00:00:00Z
    title:                             Chlorophyll-a, Aqua MODIS, NPP, L3SMI,...
    Westernmost_Easting:               -179.9792

In [10]:
print(f"Tamaño del dataset: {fuente_ds.nbytes/1e6} MB")

Tamaño del dataset: 36578.3578 MB


In [11]:
año_inicio = 2005
año_final= 2010

In [12]:
subset_fuente_ds = fuente_ds.sel(
    time=slice(datetime.datetime(año_inicio,1,1), datetime.datetime(año_final,12,31)), 
    latitude=slice(ecoregion_bordes['maxy'], ecoregion_bordes['miny']),
    longitude=slice(ecoregion_bordes['minx'], ecoregion_bordes['maxx'])
)

In [13]:
subset_fuente_ds

<xarray.Dataset>
Dimensions:    (time: 72, latitude: 198, longitude: 193)
Coordinates:
  * time       (time) datetime64[ns] 2005-01-16 2005-02-16 ... 2010-12-16
  * latitude   (latitude) float32 -25.06 -25.1 -25.15 ... -33.19 -33.23 -33.27
  * longitude  (longitude) float32 -75.1 -75.06 -75.02 ... -67.19 -67.15 -67.1
Data variables:
    chlor_a    (time, latitude, longitude) float32 ...
Attributes: (12/52)
    _lastModified:                     2023-01-31T21:02:35.000Z
    cdm_data_type:                     Grid
    Conventions:                       CF-1.6 ACDD-1.3, COARDS
    creator_email:                     data@oceancolor.gsfc.nasa.gov
    creator_name:                      NASA/GSFC/OBPG
    creator_type:                      group
    ...                                ...
    summary:                           Moderate Resolution Imaging Spectrorad...
    temporal_range:                    month
    time_coverage_end:                 2022-11-16T00:00:00Z
    time_coverage_start:               2002-07-16T00:00:00Z
    title:                             Chlorophyll-a, Aqua MODIS, NPP, L3SMI,...
    Westernmost_Easting:               -179.9792

In [14]:
print(f"Tamaño del dataset: {subset_fuente_ds.nbytes/1e6} MB")

Tamaño del dataset: 11.007772 MB


In [15]:
%%time
subset_fuente_ds.load();

CPU times: user 231 ms, sys: 185 ms, total: 416 ms
Wall time: 7min 5s


In [16]:
subset_fuente_ds

<xarray.Dataset>
Dimensions:    (time: 72, latitude: 198, longitude: 193)
Coordinates:
  * time       (time) datetime64[ns] 2005-01-16 2005-02-16 ... 2010-12-16
  * latitude   (latitude) float32 -25.06 -25.1 -25.15 ... -33.19 -33.23 -33.27
  * longitude  (longitude) float32 -75.1 -75.06 -75.02 ... -67.19 -67.15 -67.1
Data variables:
    chlor_a    (time, latitude, longitude) float32 0.1504 0.1458 ... nan nan
Attributes: (12/52)
    _lastModified:                     2023-01-31T21:02:35.000Z
    cdm_data_type:                     Grid
    Conventions:                       CF-1.6 ACDD-1.3, COARDS
    creator_email:                     data@oceancolor.gsfc.nasa.gov
    creator_name:                      NASA/GSFC/OBPG
    creator_type:                      group
    ...                                ...
    summary:                           Moderate Resolution Imaging Spectrorad...
    temporal_range:                    month
    time_coverage_end:                 2022-11-16T00:00:00Z
    time_coverage_start:               2002-07-16T00:00:00Z
    title:                             Chlorophyll-a, Aqua MODIS, NPP, L3SMI,...
    Westernmost_Easting:               -179.9792

* Escoger o mas bien trabajar con las estaciones de Chile, que son:
primavera del 23 de septiembre al 21 de diciembre
verano del 21 de diciembre y al 20 de marzo
otoño20 de marzo y al 21 de junio
invierno del 21 de junio al 23 de septiembre

La libreria que puede hacer este tipo de compuestos es
* groupby 